## Create the Python venv

```
python -V            # 3.10/3.11
python -c "import platform; print(platform.machine())"  # arm64
pip install -U pip wheel setuptools
```

# Install native ARM64 wheels

```
pip install torch torchvision torchaudio accelerate
```

## Verify PyTorch MPS (Apple GPU) works

You should see `MPS available: True`and tensors on `mps:0`

In [1]:
import torch, platform, sys
print("Python:", sys.version)
print("Arch:", platform.machine())
print("Torch:", torch.__version__)
print("MPS available:", torch.backends.mps.is_available())
print("MPS built:", torch.backends.mps.is_built())

if torch.backends.mps.is_available():
    x = torch.rand(2,2, device="mps")
    y = x @ x
    print("OK on MPS:", y.device, y)

Python: 3.10.8 (v3.10.8:aaaf517424, Oct 11 2022, 10:14:40) [Clang 13.0.0 (clang-1300.0.29.30)]
Arch: arm64
Torch: 2.8.0
MPS available: True
MPS built: True
OK on MPS: mps:0 tensor([[0.6776, 0.7412],
        [1.0213, 1.1920]], device='mps:0')


In [3]:
from accelerate import Accelerator
acc = Accelerator()
print(acc.device)  

mps


## What we did an what to check

1. ✅ Created Python venv (3.10, arm64)
2. ✅ Installed ARM64 wheels (`torch`, `accelerate`)
3. ✅ Verified PyTorch MPS: `is_built=True`, `is_available=True`
4. ✅ Ran `accelerate config` → chose:

   * This machine
   * No distributed training
   * CPU only? → **NO** (so MPS is used)
   * Torch dynamo → **NO**
   * Mixed precision → **NO**
   * No DeepSpeed
5. ✅ Config saved, `Accelerator().device` should now return **mps**

👉 You now have a working local RAG dev env on Mac, using Apple GPU.


## Handy commands:

```bash
poetry config virtualenvs.in-project true   # optional: keep .venv in repo folder
poetry env use 3.10
poetry install                               # installs from poetry.lock
poetry add torch accelerate typer llama-cpp-python hnswlib sentence-transformers
```

If you ever need a `requirements.txt` (e.g., Docker/CI without Poetry):

```bash
poetry export -f requirements.txt -o requirements.txt --without-hashes
# or include dev deps:
poetry export -f requirements.txt -o requirements-dev.txt --with dev --without-hashes
```
